In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import util

### Lecture 9 - Part 1

# Data Imputation

## Outline
* Review: How are data missing?
* Single Valued Imputation
* Probabilistic Imputation (drawing from the distribution)

### Review: Permutation Tests

* Given two observed groups of data, are they from different distributions?
* **Example**: Given a list of people's heights, are heights of the males significantly different from females?
* Permutation test assesses the likelihood the difference in average height of each group is significantly different.

In [ ]:
heights = pd.read_csv('./heights.csv')
heights = (
    heights
    .rename(columns={'childHeight': 'child', 'childNum': 'number'})
    .drop('midparentHeight', axis=1)
)
heights.head()

In [ ]:
heights.groupby('gender').child.plot(kind='kde', legend=True, title='distribution of heights by gender');

In [ ]:
distr, obs = util.permutation_test(heights, 'child',  'gender', util.diff_in_means)

pd.Series(distr).plot(kind='hist', title='permutation test: heights by gender')
plt.scatter(obs, 1, color='r', s=10);

### Review: MCAR/MAR/NMAR

* **MCAR**: Data is *Missing Completely at Random* if there is no relationship between the missingness of the data and any values, observed or missing.

* **MAR**:  Data is *Missing at Random* if there is a systematic relationship between the propensity of missing values and the observed data, but not the missing data. 

* **NMAR**: Data is *Not Missing at Random* if there is a relationship between the propensity of a value to be missing and its values. 

### Review: MCAR/MAR/NMAR  + think of examples

* **MCAR**: Data is *Missing Completely at Random* if 

* **MAR**:  Data is *Missing at Random* if 

* **NMAR**: Data is *Not Missing at Random* if 

### Example: Missingness of child height on father height (`MCAR`)

* To determine if the missingness of `child` height is dependent on height of `father`:
    - Check if the distribution of `father` heights looks the same when `child` is null vs. not-null.
    - Use a permutation test to check this!
* To assert that `child` is likely MCAR (unconditionally ignorable), the missingness of `child` height should *not* be dependent on *any* other column.

In [ ]:
# make an MCAR dataset
heights_mcar = util.make_mcar(heights, 'child', pct=0.5)
heights_mcar.isnull().mean()

In [ ]:
(
    heights_mcar
    .assign(is_null=heights_mcar.child.isnull())
    .groupby('is_null')
    .father
    .plot(kind='kde', legend=True, title='father heights by child=NULL vs child=NOT NULL')
);

In [ ]:
# run a permutation test with K-S statistic
# randomly shuffle missing/not missing
distr, obs = util.permutation_test(
    heights_mcar.assign(is_null=heights_mcar.child.isnull()), 
    'father',  'is_null', util.ks)

In [ ]:
pval = (distr >= obs).mean()
print('p-value: %f' % pval)
pd.Series(distr).plot(kind='hist', title='permutation test for father heights by child=NULL vs child=NOT NULL')
plt.scatter(obs, 5, color='r', s=20, zorder=10);

## Discussion Question

If we take the mean of child heights (dropping NaNs), is the result likely to:

1. overestimate the true mean
2. underestimate the true mean
3. be accurate

In [ ]:
(
    heights_mcar
    .assign(is_null=heights_mcar.child.isnull())
    .groupby('is_null')
    .father
    .plot(kind='kde', legend=True, title='father heights by child=NULL vs child=NOT NULL')
);

### Example: Missingness of child height on father height (Conditionally Ignorable: `MAR`)

* To determine if the missingness of `child` height is dependent on height of `father`:
    - Check if the distribution of `father` heights looks the same when `child` is null vs. not-null.
    - Use a permutation test to check this!

In [ ]:
# dependency of the missingness on child by father
# 75% of the data is missing
heights_mar = util.make_mar_on_num(heights, 'child', 'father', pct=0.75)
heights_mar.isnull().mean()

In [ ]:
(
    heights_mar
    .assign(is_null=heights_mar.child.isnull())
    .groupby('is_null')
    .father
    .plot(kind='kde', legend=True, title='father heights by child=NULL vs child=NOT NULL')
);
# taller fathers are less likely to report the height if their child
# when child hight is missing, the mean father hight is higher

In [ ]:
distr, obs = util.permutation_test(
    heights_mar.assign(is_null=heights_mar.child.isnull()), 
    'father',  'is_null', util.ks)

In [ ]:
pval = (distr >= obs).mean()
print('p-value: %f' % pval)
pd.Series(distr).plot(kind='hist', title='permutation test for father heights by child=NULL vs child=NOT NULL')
plt.scatter(obs, 0, color='r', s=10);

## Discussion Question

If we take the mean of child heights (dropping NaNs), is the result likely to:

1. overestimate the true mean
2. underestimate the true mean
3. be accurate

In [ ]:
(
    heights_mar
    .assign(is_null=heights_mar.child.isnull())
    .groupby('is_null')
    .father
    .plot(kind='kde', legend=True, title='father heights by child=NULL vs child=NOT NULL')
);
# taller fathers are less likely to report the height if their child
# when child hight is missing, the mean father hight is higher

In [ ]:
# if you want to predict a child height from father height
# taller fathers tend to have taller children 
# we will underestimate the height of the children in our predictions
# there is a pattern to the missing data: children height is missing more often 
# when fathers were tall

### Part 2

# What to do with missing data?


## What to do with missing data?

* Given a dataset $Y$ with observed values $Y_{obs}$ and missing values $Y_{mis}$.
* $Y$ may appear quite different than $Y_{obs}$:
    - The mean and percentile may be different.
    - The variance may be different.
    - Correlations between variables may be different.

## Example

- Survey with optional question: how much do you give to charity?
- People who give little are less likely to respond.
- Therefore, average response is **biased high**.

## Simple ways of dealing with missing data

If the data are ______, `.dropna()` doesn't significantly change the data. E.g., it produces an unbiased estimate of the mean.

* MCAR (unconditionally ignorable / missing completely at random)

* MAR (conditionally ignorable / missing at random)

* NMAR (nonignorable / not missing at random)

* None of the above

## Key Point

- How we handle missing data depends on the *type of missingness*.
- Filling in missing values without taking the type of missingingness into account will likely bias your dataset.
- Diminish the usefulness of your results.

## Ways of dealing with missing data: deletion

Using *listwise deletion* (dropping observations (rows)) to remove missing values:
* Results in unbiased means only when the data are *MCAR*. (rare)
* Removes observations with non-null data in other fields. (bad)
    - Improvement: can drop missing data only when computing with the column in question.
* Doesn't further bias the observed dataset.

Explain to yourself why these are true! (Recall DSC 10)

In [ ]:
np.random.seed(42)
heights_mcar = util.make_mcar(heights, 'child', pct=0.50)
heights_mar = util.make_mar_on_cat(heights, 'child', 'gender', pct=0.50)

In [ ]:
# the "correct" mean
heights['child'].mean()

In [ ]:
# the mean of the column with data MCAR
heights_mcar['child'].dropna().mean()

In [ ]:
# .mean() automatically drops NaNs!!!
heights_mcar['child'].mean()

In [ ]:
# the mean with data MAR
heights_mar['child'].mean()

## Ways of dealing with missing data: imputation

**Imputation** is the act of filling in missing data with plausable values.

* Should be quick and easy to do
* Shouldn't "change" the dataset

These are hard to satisfy!

## Three kinds of imputation

* imputation with a single value: mean, median, mode
* imputation with a single value, using a model: regression, kNN
* (probabilistic) imputation by drawing from a distribution

Each has upsides and downsides; each works differently with different types of missingness.

### Part 3

# Mean imputation

## Mean imputation

* Imputing a missing value with the mean:
    - preserves the mean of the observed data
        - no matter the type of missingness
        - 👍
    - decreases the variance
        - no matter the type of missingness
        - 👎
    - biases the mean
        - when not *MCAR*
        - 👎

## Mean imputation in the `heights` data
* Mean imputation of MCAR data
    - unbiased estimator of the mean.
    - decreases variance.

In [ ]:
# counts of child heights
plt.hist([heights_mcar.child.dropna(), heights.child])
plt.legend(['missing (mcar)', 'full data']);

In [ ]:
# impute with mean
heights_mcar_mfilled = heights_mcar.fillna(heights_mcar.child.mean())

In [ ]:
print(
    'mean (original): %f' % heights.child.mean(),
    'mean (missing):  %f' % heights_mcar.child.mean(),
    'mean (mean imp): %f' % heights_mcar_mfilled.child.mean(),
    sep='\n'
)

In [ ]:
# Why is this smaller, given formula for std
print(
    'std (original): %f' % heights.child.std(),
    'std (missing):  %f' % heights_mcar.child.std(),
    'std (mean imp): %f' % heights_mcar_mfilled.child.std(),
    sep='\n'
)

### Mean imputation of MCAR data

* Unbiased estimator of mean.
* Variance is drastically reduced.
    - Tricks you into thinking your data are more reliable than they are!

In [ ]:
# counts of child heights
plt.hist([heights.child, heights_mcar.child.dropna(), heights_mcar_mfilled.child])
plt.legend([ 'full data', 'missing (mcar)', 'imputed']);

## Mean imputation and MAR data

* Naive mean imputation leads to biased estimates of mean across groups, when using MAR data.
    - e.g., if missingness depends on gender, different genders will have differently-biased means
    - overall mean will be biased towards one group


In [ ]:
np.random.seed(42)
heights_mar_cat = util.make_mar_on_cat(heights, 'child', 'gender', pct=0.50)

In [ ]:
# The observed vs true distribution
plt.hist([heights.child, heights_mar_cat.child]);
plt.legend([ 'full data','missing (mar)']);

In [ ]:
# naive mean imputation
heights_mar_cat_mfilled = heights_mar_cat.fillna(heights_mar_cat.child.mean())

In [ ]:
print(
    'mean (original): %f' % heights.child.mean(),
    'mean (missing):  %f' % heights_mar_cat.child.mean(),
    'mean (mean imp): %f' % heights_mar_cat_mfilled.child.mean(),
    sep='\n'
)

In [ ]:
print(
    'std (original): %f' % heights.child.std(),
    'std (missing):  %f' % heights_mar_cat.child.std(),
    'std (mean imp): %f' % heights_mar_cat_mfilled.child.std(),
    sep='\n'
)

In [ ]:
plt.hist([heights.child, heights_mar_cat.child, heights_mar_cat_mfilled.child]);
plt.legend([ 'full data','missing (mar)', 'imputed']);

## Within-group mean imputation

- a.k.a., Conditional mean imputation
* Since MAR is MCAR within each group, can do group-wise mean imputation.
    - e.g., replace missing height by calculating the mean height within same gender
- unbiased overall mean, unbiased group means
- reduces variance

In [ ]:
# Biased mean by groups!
# How is this connected to simpson's paradox?
pd.concat([
    heights.groupby('gender').child.mean().rename('full'),
    heights_mar_cat.groupby('gender').child.mean().rename('missing (mar)'),
    heights_mar_cat_mfilled.groupby('gender').child.mean().rename('naively imputed')
], axis=1)

### Discussion Question

* In MAR data, imputation by the mean gives a biased estimate of the mean of each group. 
* You can obtain an unbiased estimate of means across groups by imputing with the mean within each group.
* How would you code this up for `child` heights that are MAR dependent on `gender`?

Recall that unconditional mean imputation is given by `heights.child.fillna(heights.child.mean())`

## Conclusions: imputation with single values
* Imputing missing data in a column with the mean of the column:
    - faithfully reproduces the mean of the observed dataset,
    - reduces the variance,
    - biases relationships of the column with other columns (if not MCAR).
    
* Similar with other statistics (median, mode).

### Discussion Question

* Consider the income reporting in the US Census. 
* Suppose we impute missing salaries with the mean overall income.
* Is there more bias in:
    - (low-paying) service jobs or 
    - (high-paying) executive jobs?
    
Hint: what does the distribution of incomes look like? Where is the mean/median?

### Part 4

# Imputing using distributions

## Imputing missing values using distributions
* We can *probalistically* impute missing data from a distribution.
    - Fill in missing data by drawing from the distribution of *non-missing* data.
    - I.e., we need to fill in this missing value. Idea: just randomly choose an observed value.
* Recall: using `.sample`, we can draw from an (empirical) distribution.

### Imputing `heights_mcar` data by drawing from a distribution.

Steps:
* Sample values of child heights from the observed heights.
* Fill in missing child heights by using these draws from the observed heights.

In [ ]:
heights_mcar_dfilled = ...

In [ ]:
num_null = heights_mcar.child.isnull().sum() # number of nulls
fill_values = heights_mcar.child.dropna().sample(num_null, replace=True)  # draw fill vals from distribution
fill_values.index = heights_mcar.loc[heights_mcar.child.isnull()].index  # align the index
heights_mcar_dfilled = heights_mcar.fillna({'child': fill_values.to_dict()})  # fill the vals

In [ ]:
print(
    'mean (original):  %f' % heights.child.mean(),
    'mean (missing):   %f' % heights_mcar.child.mean(),
    'mean (distr imp): %f' % heights_mcar_dfilled.child.mean(),
    sep='\n'
)

In [ ]:
print(
    'std (original):  %f' % heights.child.std(),
    'std (missing):   %f' % heights_mcar.child.std(),
    'std (distr imp): %f' % heights_mcar_dfilled.child.std(),
    sep='\n'
)

In [ ]:
plt.hist([heights.child, heights_mcar.child, heights_mcar_dfilled.child], density=True);
plt.legend([ 'full data','missing (mcar)', 'distr imputed']);

## Observations
* We only drew missing values from observed values. What if a missing value was previously unobserved?
    - better to bin the data and draw from bins instead! 
    - use `np.histogram` to bin data.

* How does this generalize to categoricals?

## Observations

* We created an imputed dataset using randomness to preserve variance
* Our imputation could have been different, had we run it again
* *Multiple imputation*: generate multiple imputed datasets and aggregate each result!
    - Should remind you of bootstrap estimation.

## Multiple Imputation (Donald Rubin)

Multiple imputation is a 3 step process:

<div class="image-txt-container">

* **Imputation**: Impute a missing data multiple times (m times)
* **Analysis**: Analyze each complete dataset separetly (m sets)
* **Pooling**: Combine multiple analysis result. M estimates result in the final estimate.

<img src="imgs/mult_imp.png" width="50%">

</div>


### Multiple Imputation: steps

0. Start with observed and incomplete data. 
1. Create several **imputed** versions of the data by drawing from a distribution of plausible values.
    - The imputed datasets are identical for the observed data entries, 
    - they differ in the imputed values. 
    - The differences reflect our **uncertainty** about what value to impute.

2. Then we estimate the parameters of interest from **each** imputed dataset.
3. The last step is to pool the m parameter estimates into one estimate and to estimate its variance.

In [ ]:
# heights_mcar is our incomplete data

def create_imputed(col):
    num_null = col.isnull().sum()
    fill_values = col.dropna().sample(num_null, replace=True)
    fill_values.index = col.loc[col.isnull()].index
    return col.fillna(fill_values.to_dict())


In [ ]:
create_imputed(heights_mcar.child).head()

In [ ]:
mult_imp = pd.concat([create_imputed(heights_mcar.child).rename(k) for k in range(100)], axis=1)
mult_imp.head()

In [ ]:
# plot 15 random imputations
mult_imp.sample(15, axis=1).plot(kind='kde', alpha=0.5, legend=False);

In [ ]:
# sampling distribution of imputed means
mult_imp.mean().plot(kind='hist', bins=20);

In [ ]:
# doesn't decrease the standard deviations of imputed data
mult_imp.std().plot(kind='hist', bins=50)
std_incomplete = heights_mcar.child.std()
plt.plot([std_incomplete, std_incomplete], [0,8]);

## Missingness, conditional on multiple variables
* Use multiple imputation, as in previous example
* Impute from a distribution for each variable
* Do an analysis per imputation; aggregate the analyses.

## Summary of Imputation Techniques

* Listwise deletion
* Mean imputation
* Conditional (groupwise) mean imputation
* Probabilistic imputation
* Multiple imputation

## Summary: listwise deletion

* Procedure: `.dropna()`
* If MCAR, doesn't change statistics of the data
* If MCAR and small, may have high variance

## Summary: (single value) mean imputation 

* Procedure: `.fillna(dataframe[col].mean())`
* If MCAR, gives unbiased estimate of mean; variance is too low.
* Analogue for categorical data: imputation with the mode.

## Summary: conditional mean imputation

* Procedure: for a column `c1`, conditional on a second column `c2`:
```
means = dataframe.groupby('c2').mean().to_dict()
imputed = dataframe['c1'].apply(lambda x: means[x] if pd.isnull(x) else x)
```
* If MAR, gives unbiased estimate of mean; variance is too low.
* Increases correlations between the columns.
* If dependent on *more than one* column: use linear regression to predict missing value.

## Summary: probabilistic imputation

* Procedure: draw from empirical distribution of observed data to fill missing values.
* If MCAR, gives unbiased estimate of mean and variance.
* Extending to MAR case: draw from *conditional empirical distributions*
    - if conditional on a single categorical column `c2`: 
    - apply MCAR procedure to the groups of `dataframe.groupby(c2)`

## Summary: multiple imputation

* Procedure:
    - Apply probabilistic imputation multiple times, resulting in $N$ imputed datasets.
    - Do analyses separately on the $N$ imputed datasets (e.g. compute correlation coefficient).
    - Plot the distribution of the results of these analyses!
* If a column is missing conditional on multiple columns, your "multiple imputations" should include probabilistic imputations for each!

## What about MAR?

- Missing value depends on other columns.
- Idea: try to predict it.
- We'll cover this when we talk about models in a few weeks.

## Summary of Imputation Techniques


|---|Unbiased|---|---|Standard Error|
|---|---|---|---|---|
|---|Mean|Regression weights|Correlation|---|
|Listwise deletion|MCAR|MCAR|MCAR|Large|
|Mean|MCAR|---|---|Small|
|Regressions|MAR|MAR|---|Small|
|Probabilistic|MAR|MAR|MAR|Large|
|Multiple|MAR|MAR|MAR|Good|
